# MEModel With Synapses Simulation Example with OBI-One Form Logic

This notebook demonstrates how to run a MEModel With Synapses simulation using the OBI-One form-based workflow, similar to the circuit simulation example.

In [ ]:
from entitysdk import Client
from obi_auth import get_token

from obi_notebook import get_projects

obi_one_api_url = "http://127.0.0.1:8100"

token = get_token(environment="production", auth_mode="daf")
project_context = get_projects.get_projects(token)

db_client = Client(environment="production", project_context=project_context, token_manager=token)

In [4]:
entity_ID = "eca93906-0331-48ef-994b-1209ecc8eb6c"

In [ ]:
import obi_one as obi

sim_duration = 3000.0

sim_form = obi.MEModelWithSynapsesCircuitSimulationScanConfig.empty_config()

# Info
info = obi.Info(
    campaign_name="MEModel With Synapses Simulation",
    campaign_description="Simulation of example MEModel With Synapses (with morphology locations)",
)
sim_form.set(info, name="info")

# Timestamps
timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=100)
sim_form.add(timestamps, name="Timestamps")

# Stimulus 1: Poisson
poisson = obi.PoissonSpikeStimulus(
    duration=800.0,
    timestamps=timestamps.ref,
    frequency=20,
)
sim_form.add(poisson, name="PoissonInput")

# --- Locations rule ---
loc_block = obi.RandomMorphologyLocations(
    random_seed=0,
    number_of_locations=2,
    section_types=(3, 4),
)
sim_form.add(loc_block, name="ClampLocations")

# Stimulus 2: current clamp that targets generated locations
clamp = obi.ConstantCurrentClampStimulus(
    timestamps=timestamps.ref,
    duration=2000.0,
    amplitude=0.5,
    locations=loc_block.ref,
)
sim_form.add(clamp, name="CurrentClampInput")

# Recording
recording = obi.SomaVoltageRecording()
sim_form.add(recording, name="SomaVoltage")

# Initialization block
init = obi.MEModelWithSynapsesCircuitSimulationScanConfig.Initialize(
    circuit=obi.MEModelWithSynapsesCircuitFromID(id_str=entity_ID),
    simulation_length=sim_duration,
)
sim_form.set(init, name="initialize")

# Validate
validated_sim_conf = sim_form.validated_config()
print(validated_sim_conf)

# Run
grid_scan = obi.GridScanGenerationTask(
    form=validated_sim_conf,
    coordinate_directory_option="ZERO_INDEX",
    output_root="./obi-output/run_circuit_simulations/grid_scan",
)
grid_scan.execute(db_client=db_client)
obi.run_tasks_for_generated_scan(grid_scan, db_client=db_client)

In [6]:
simulation_config_path = grid_scan.single_configs[0].coordinate_output_root / "simulation_config.json"
print(simulation_config_path)

circuit_folder = grid_scan.single_configs[0].coordinate_output_root / "sonata_circuit"

obi-output/run_circuit_simulations/grid_scan/0/simulation_config.json


In [ ]:
# Remove the old compiled mod files folder
! rm -r arm64/
# flag DISABLE_REPORTINGLIB to skip SonataReportHelper.mod and SonataReport.mod from compilation.
!../../.venv/bin/nrnivmodl -incflags "-DDISABLE_REPORTINGLIB" {circuit_folder}/mod

In [ ]:
# === 4. Run the simulation (BlueCelluLab backend) ===
from obi_one.scientific.library.simulation_execution import run

run(
    simulation_config=simulation_config_path,
    simulator='bluecellulab',
    save_nwb=False
)

## Results
The results are stored in the `output` directory. You can analyze the voltage traces and other outputs as needed.

In [9]:
import bluepysnap
snap_simulation = bluepysnap.Simulation(simulation_config_path)
spikes = snap_simulation.spikes
print(
    spikes.time_start,
    spikes.time_stop,
    spikes.dt
)
print(spikes.population_names)

0 3000.0 0.025
['S1nonbarrel_neurons']


In [10]:
spike_pop = spikes[spikes.population_names[0]]
node_population = spike_pop.nodes
filtered = spikes.filter( t_start=spikes.time_start, t_stop=spikes.time_stop)
filtered.report.head()

,ids,population
times,,
1.000000e-10,0,S1nonbarrel_neurons
1.135000e+01,0,S1nonbarrel_neurons
2.075000e+01,0,S1nonbarrel_neurons
3.010000e+01,0,S1nonbarrel_neurons
3.960000e+01,0,S1nonbarrel_neurons


In [11]:
snap_simulation.reports

{'SomaVoltage': <bluepysnap.frame_report.SomaReport at 0x12ee25310>}

In [12]:
soma_report = snap_simulation.reports['SomaVoltage']
print(
    soma_report.time_start,
    soma_report.time_stop,
    soma_report.dt
)  # Gives a warning in case the dt differs from simulation.dt

[2026-01-14 10:29:01,661] WARNING: dt from the report differs from the global simulation dt.
0.0 3000.0 0.1
